In [1]:
import bs4
import requests
from lxml import etree
import numpy as np
import spacy
from urllib.request import urlopen
from tqdm import tqdm 
import nltk
from bs4 import Doctype, NavigableString
import json
nltk.download('punkt')

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Gibson, Text

In [2]:
sites = [
    "https://math.illinois.edu/directory/profile/sahlgren",
    "https://math.illinois.edu/directory/profile/palbin",
    "https://math.illinois.edu/directory/profile/mando",
    "https://math.illinois.edu/directory/profile/jobal",
    "https://math.illinois.edu/directory/profile/ymb",
    "https://math.illinois.edu/directory/profile/danbe",
    "https://math.illinois.edu/directory/profile/fboca",
    "https://math.illinois.edu/directory/profile/bradlow",
    "https://math.illinois.edu/directory/profile/bronski",
    "https://math.illinois.edu/directory/profile/rdeville",
    "https://math.illinois.edu/directory/profile/psdey",
    "https://math.illinois.edu/directory/profile/philippe",
    "https://math.illinois.edu/directory/profile/csdodd2",
    "https://math.illinois.edu/directory/profile/nmd",
    "https://math.illinois.edu/directory/profile/s-dutta",
    "https://math.illinois.edu/directory/profile/duursma",
    "https://math.illinois.edu/directory/profile/berdogan",
    "https://math.illinois.edu/directory/profile/rfeng",
    "https://math.illinois.edu/directory/profile/ford126",
    "https://math.illinois.edu/directory/profile/rguzma1",
    "https://math.illinois.edu/directory/profile/whaboush",
    "https://math.illinois.edu/directory/profile/jbheller",
    "https://math.illinois.edu/directory/profile/aimo",
    "https://math.illinois.edu/directory/profile/hirani",
    "https://math.illinois.edu/directory/profile/verahur",
    "https://math.illinois.edu/directory/profile/ivanov",
    "https://math.illinois.edu/directory/profile/xjing4",
    "https://math.illinois.edu/directory/profile/mjunge",
    "https://math.illinois.edu/directory/profile/katzs",
    "https://math.illinois.edu/directory/profile/rinat",
    "https://math.illinois.edu/directory/profile/ekerman",
    "https://math.illinois.edu/directory/profile/kkirkpat",
    "https://math.illinois.edu/directory/profile/ekirr",
    "https://math.illinois.edu/directory/profile/kostochk",
    "https://math.illinois.edu/directory/profile/denka",
    "https://math.illinois.edu/directory/profile/lanave",
    "https://math.illinois.edu/directory/profile/laugesen",
    "https://math.illinois.edu/directory/profile/leditzky",
    "https://math.illinois.edu/directory/profile/lerman",
    "https://math.illinois.edu/directory/profile/xcli",
    "https://math.illinois.edu/directory/profile/yyyliu",
    "https://math.illinois.edu/directory/profile/ruiloja",
    "https://math.illinois.edu/directory/profile/rmccrthy",
    "https://math.illinois.edu/directory/profile/mineyev",
    "https://math.illinois.edu/directory/profile/inik",
    "https://math.illinois.edu/directory/profile/oikhberg",
    "https://math.illinois.edu/directory/profile/jpascale",
    "https://math.illinois.edu/directory/profile/zquan",
    "https://math.illinois.edu/directory/profile/zrapti",
    "https://math.illinois.edu/directory/profile/rezk",
    "https://math.illinois.edu/directory/profile/reznick",
    "https://math.illinois.edu/directory/profile/rsong",
    "https://math.illinois.edu/directory/profile/r-sowers",
    "https://math.illinois.edu/directory/profile/vesna",
    "https://math.illinois.edu/directory/profile/jat9",
    "https://math.illinois.edu/directory/profile/tolman",
    "https://math.illinois.edu/directory/profile/tumanov",
    "https://math.illinois.edu/directory/profile/tyson",
    "https://math.illinois.edu/directory/profile/tzirakis",
    "https://math.illinois.edu/directory/profile/weiw",
    "https://math.illinois.edu/directory/profile/ayong",
    "https://math.illinois.edu/directory/profile/zaharesc",
    "https://math.illinois.edu/directory/profile/vzh"
]

In [ ]:
print(len(sites))

63


## First Iter

In [ ]:
W = 5
freq = dict()
hash2elem = dict()

for url_idx in tqdm(range(len(sites))):
  url = sites[url_idx]
  html = urlopen(url).read()
  soup = bs4.BeautifulSoup(html, features="html.parser")

  for script in soup(["script"]):
      script.extract()

  # text = str(soup)
  text = soup.get_text()
  text = nltk.tokenize.word_tokenize(text)
  windows = [tuple(text[i:i+W]) for i in range(len(text) - W)]

  for window in windows:
    hsh = hash(window)
    hash2elem[hsh] = window

    if not hsh in freq:
      freq[hsh] = 0
    freq[hsh] += 1

100%|██████████| 63/63 [01:03<00:00,  1.01s/it]


In [ ]:
print(len(freq))

16606


In [ ]:
F = 10

template_hashes = set()

for hsh in freq:
  if freq[hsh] >= F:
    template_hashes.add(hsh)

In [ ]:
print(len(template_hashes))

220


## Second Iter

In [ ]:
clusters = set()

for url_idx in tqdm(range(len(sites))):
  url = sites[url_idx]
  html = urlopen(url).read()
  soup = bs4.BeautifulSoup(html, features="html.parser")

  # kill all script elements
  for script in soup(["script"]):
      script.extract()

  # text = str(soup)
  text = soup.get_text()
  text = nltk.tokenize.word_tokenize(text)
  windows = [tuple(text[i:i+W]) for i in range(len(text) - W)]

  curr_cluster = None
  for window in windows:
    hsh = hash(window)

    if hsh in template_hashes:
      if curr_cluster == None:
        curr_cluster = list(window)
      else:
        curr_cluster.append(list(window)[-1])

    else:
      if not curr_cluster == None:
        cluster_hsh = hash(tuple(curr_cluster))
        clusters.add(cluster_hsh)
        curr_cluster = None

100%|██████████| 63/63 [01:03<00:00,  1.00s/it]


In [ ]:
print(len(clusters))

34


In [ ]:
url = "https://math.illinois.edu/directory/profile/tzirakis"
html = urlopen(url).read()
soup = bs4.BeautifulSoup(html, features="html.parser")

# kill all script elements
for script in soup(["script"]):
    script.extract()

# text = str(soup)
text = soup.get_text()
text = nltk.tokenize.word_tokenize(text)

without_template = []
i = 0
while i < len(text):
  template_end = None
  for j in range(i + 1, len(text)):
    window_lst = text[i:j]
    window_tpl = tuple(window_lst)
    hsh = hash(window_tpl)

    if hsh in clusters:
      template_end = j
      break
    
  if template_end == None:
    without_template.append(text[i])
    i += 1
  else:
    i = template_end + 1

In [ ]:
print(without_template)

['Nikolaos', 'Tzirakis', 'Tzirakis', 'Contact', 'InformationDepartment', 'of', 'Mathematics', '327', 'CAB', 'tzirakis', 'Your', 'Profile', 'Professor', 'Research', 'DescriptionMy', 'research', 'area', 'is', 'Harmonic', 'Analysis', 'and', 'Partial', 'Differential', 'Equations', '.', 'More', 'precisely', 'I', 'am', 'studying', 'the', 'long', 'time', 'behavior', 'and', 'the', 'regularity', 'properties', 'of', 'dispersive', 'partial', 'differential', 'equations', ',', 'E.', ',', '&', 'Tzirakis', ',', 'N.', '(', '2020', ')', '.', 'Sharp', 'well-posedness', 'for', 'the', 'generalized', 'KdV', 'of', 'order', 'three', 'on', 'the', 'half', 'line', '.', 'Physica', 'D', ':', 'Nonlinear', 'Phenomena', ',', '402', ',', '[', '132208', ']', '.', 'https', ':', '//doi.org/10.1016/j.physd.2019.132208Erdoğan', ',', 'M.', 'B.', ',', 'Gürel', ',', 'T.', 'B.', ',', '&', 'Tzirakis', ',', 'N.', '(', '2020', ')', '.', 'The', 'fifth', 'order', 'KP-II', 'equation', 'on', 'the', 'upper', 'half-plane', '.', 'Diffe